In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
!rm output/dynamic_delta_fits/??/*
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 1
sim = 0
new = 1
Min_Entries = 100
N_files = 12

In [3]:
isigma = 2
N_layers = 16
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if new: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NewHadron/rungroup/"
file_names="my-12th_QA_19423_1078runs_rg" #my-25th_QA_19197_1000runs
if not new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [4]:
if sim :
    file_path="../sim/output/"
    file_names="my-kek"

In [5]:
hists_read = []

for iFile in range(-1,N_files):
    if new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_read0.append(hist_select_3D0)
    hists_read.append(hists_read0)

infile.Close()

In [6]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.35, 0.65, 1.0, 1.4, 1.8, 2.2, 2.7, 3.4, 4.35]) 
pt_boarders =  array('d', [0.2, 0.5, 0.8, 1.2, 1.6, 2.0, 2.4, 3.0, 3.8, 4.9])


In [7]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]
arms = N_layers/8

In [8]:
def sdphi_fitter(InHist, iphi, iFile=0, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0, accur = 0.5):
    
    N_gaus=2
    seed0=iFile*N_pt*ilayer*100
    iilayer = int(ilayer/arms)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if new: central_bean_prev = (0.002*phi_bins[2*iphi]-0.001-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed0+p}",f"c1{seed0+p}",720,360)

        m2_name=str(ilayer)+"/"+"F"+str(iFile)+"_P"+str(iphi)+"_dynamic_"+str(round(pt[p],1))

        var_porj = InHist.ProjectionX(f"emc{seed0+p}_{seed0+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        var_porj.SetLineColor(1)

        if var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed0+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed0+p}_{ilayer}_{iphi}', 'gaus',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed0+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed0+p}_{ilayer}_{iphi}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)

        if fits_mean == 0:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[1][1])
            g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
            g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
            g_gaus2.SetParameter(1,0.5*(boards_fit2[iilayer][0]+boards_fit2[iilayer][1])+central_bean_prev)
            g_gaus2.SetParameter(2,0.5*(boards_fit2[iilayer][1]-boards_fit2[iilayer][0]))
            g_gaus2.SetParLimits(0,1,10000000)
            g_gaus2.SetParLimits(1,boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)
        else: 
            mean0 = fits_mean[0][ilayer][iphi].Eval(pt_loc)
            sigm0 = fits_sigma[0][ilayer][iphi].Eval(pt_loc)
            g_gaus.SetParameter(1,mean0)
            g_gaus.SetParameter(2,sigm0)

        var_porj.Draw()
        var_porj.Fit( g_gaus, 'QR')

        var_porj.Fit( g_gaus2, 'QR+' )
        var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/100,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        if iOption0==1:
            total.SetParameter(3,0.)
            total.SetParLimits(3,0,par_gaus2[0]*0.01)
            
        if not fits_mean==0: 
            mean0 = fits_mean[0][ilayer][iphi].Eval(pt_loc)
            sigm0 = fits_sigma[0][ilayer][iphi].Eval(pt_loc)
            total.SetParLimits(1,mean0-0.5*sigm0,mean0+0.5*sigm0)
            total.SetParLimits(2,sigm0*0.8,sigm0*1.25)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        var_porj.Draw()
        var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed0+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed0+p}_{ilayer}_{iphi}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if is_sigmalised:
            if abs(total.GetParameter(1))>1.5:
                continue

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if is_sigmalised==0:
            Mean_local_err.append (total.GetParError(1)+0.1*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.1*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ 0.25)#(0.15-0.005*p)*total.GetParameter(2)
            Sigma_local_err.append(total.GetParError(2)+0.25)#(0.15-0.005*p)*total.GetParameter(2)


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [9]:
def sdphi_fitter_centr(ilayer=1, start_file=-1, last_file=0, fits_mean=0, fits_sigma=0,InHist=hists_read, phibins=[0,1,2,3,4,5], sigma=2):
    out_hists = []
    for iFile in range(start_file,last_file):
        out_hists1 = []
        for iphi in phibins:
            out_hists1.append(sdphi_fitter(InHist[iFile][ilayer][iOption0+is_sigmalised*2], iphi, iFile, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [10]:
def sdphi_fitter_run(ilayer=1):
    return sdphi_fitter_centr(ilayer)

In [70]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter_centr, [i for i in range(Ntr)])
pool.close()

In [71]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(1):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [72]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions_0iter = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_0iter_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.02)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions_0iter.append(fit_functions0)

In [73]:
g_sigma = []
for ifile in range(1):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{iFile}",f"c3{iFile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    Min, Max = 0., 0.01
    if is_sigmalised==1: Max, Min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        g_sigma_layer = []
        h2.append(Format_Pad_old( 0.05, 5, Min, Max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions_0iter[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions_0iter[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ilayer}{iOption0}{is_sigmalised}.png")

In [74]:
fit_mean_functions_0iter = []
for ifile in range(1):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_0iter_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_0iter_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0<2:
                    fit_functions1[-1].SetParameter(2,-1)
                    fit_functions1[-1].SetParLimits(2,-1.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-5,5)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,-5,5)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    
                    fit_functions1[-1].FixParameter(2,-1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions_0iter.append(fit_functions0)

In [75]:
g_mean = []
for iFile in range(1):
    g_mean_f = []
    h2=[]
    c3 = root.TCanvas(f"c2{iFile}",f"c2{iFile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    Min, Max = -0.05, 0.05
    if is_sigmalised==1: Max, Min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        g_mean_layer = []
        h2.append(Format_Pad_old( 0.05, 5, Min, Max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            g_mean_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_mean_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if not is_sigmalised: g_mean_layer[-1].Fit(fit_mean_functions_0iter[ifile][ilayer][iphi],"Q","",0.7,1)
            else: g_mean_layer[-1].Fit(fit_mean_functions_0iter[ifile][ilayer][iphi],"Q","",0.5,2)
            g_mean_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_mean_f.append(g_mean_layer)
    g_mean.append(g_mean_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ilayer}{iOption0}{is_sigmalised}.png")

In [76]:
def sdphi_fitter_run(ilayer=1):
    return sdphi_fitter_centr(ilayer,0,N_files,fit_mean_functions_0iter,fit_functions_0iter)

In [77]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array1 = pool.map(sdphi_fitter_run, [i for i in range(Ntr)])
pool.close()

In [78]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(N_files):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array1[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array1[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array1[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array1[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array1[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array1[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array1[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array1[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [79]:
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameters(fit_functions_0iter[0][ilayer][iphi].GetParameters())
            p0 = fit_functions_0iter[0][ilayer][iphi].GetParameter(0)
            p1 = fit_functions_0iter[0][ilayer][iphi].GetParameter(1)
            p2 = fit_functions_0iter[0][ilayer][iphi].GetParameter(2)
            #fit_functions1[-1].SetParLimits(0,p0-0.001,p0+0.001)
            #fit_functions1[-1].SetParLimits(1,-5,5)
            #fit_functions1[-1].SetParLimits(2,-5,5)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [82]:
g_sigma = []
for ilayer in range(N_layers):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3_{ilayer}",f"c3_{ilayer}",100*N_layers,1000)
    c3.Divide(int(N_files/4),4)
    Min, Max = 0., 0.01
    if is_sigmalised==1: Max, Min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ifile in range(N_files):
        g_sigma_layer = []
        c3.cd(ifile+1)
        h2.append(Format_Pad_old( 0.05, 5, Min, Max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),0.5, colors[iphi], 1, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    #c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ilayer}{iOption0}{is_sigmalised}.png")

TypeError: none of the 9 overloaded methods succeeded. Full details:
  TGraphErrors::TGraphErrors(const TVectorT<float>& vx, const TVectorT<float>& vy, const TVectorT<float>& vex, const TVectorT<float>& vey) =>
    TypeError: takes at most 4 arguments (5 given)
  TGraphErrors::TGraphErrors(const TVectorT<double>& vx, const TVectorT<double>& vy, const TVectorT<double>& vex, const TVectorT<double>& vey) =>
    TypeError: takes at most 4 arguments (5 given)
  TGraphErrors::TGraphErrors(const TGraphErrors& gr) =>
    TypeError: takes at most 1 arguments (5 given)
  TGraphErrors::TGraphErrors(const TH1* h) =>
    TypeError: takes at most 1 arguments (5 given)
  TGraphErrors::TGraphErrors() =>
    TypeError: takes at most 0 arguments (5 given)
  TGraphErrors::TGraphErrors(int n) =>
    TypeError: takes at most 1 arguments (5 given)
  TGraphErrors::TGraphErrors(const char* filename, const char* format = "%lg %lg %lg %lg", const char* option = "") =>
    TypeError: takes at most 3 arguments (5

In [57]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0<2:
                    fit_functions1[-1].SetParameter(2,-1)
                    fit_functions1[-1].SetParLimits(2,-1.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-5,5)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,-5,5)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    
                    fit_functions1[-1].FixParameter(2,-1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [58]:
g_mean = []
for ilayer in range(N_layers):
    g_mean_f = []
    h2=[]
    c3 = root.TCanvas(f"c3_{ilayer}",f"c3_{ilayer}",100*N_layers,1000)
    c3.Divide(int(N_files/4),4)
    Min, Max = -0.05, 0.05
    if is_sigmalised==1: Max, Min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ifile in range(N_files):
        g_mean_layer = []
        c3.cd(ifile+1)
        h2.append(Format_Pad_old( 0.05, 5, Min, Max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            g_mean_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_mean_layer[-1],20+int(iphi/4),0.5, colors[iphi], 1, colors[iphi],1,1)
            if not is_sigmalised: g_mean_layer[-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.7,2)
            else: g_mean_layer[-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.5,2)
            g_mean_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_mean_f.append(g_mean_layer)
    g_mean.append(g_mean_f)
    #c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean{ilayer}{iOption0}{is_sigmalised}.png")

In [59]:
dvfsdf

NameError: name 'dvfsdf' is not defined

In [66]:
#print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
#for fit_functions_ifile in fit_functions:
#    print("{")
#    for fit_functions_ilayer in fit_functions_ifile:
#        print("{")
#        for fits in fit_functions_ilayer:
#            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
#        print("},") 
#    print("},")      
#print("};")        

In [17]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_layers}][{N_phi}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iilayer = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iilayer%2)].GetParameter(0),6)}, {round(fits[3-int(iilayer%2)].GetParameter(1),6)}, {round(fits[3-int(iilayer%2)].GetParameter(2),6)}"+"},")
            iilayer+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_ilayer in fit_functions_ifile:
            print("{")
            for fits in fit_functions_ilayer:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

const float phi_sMean_pt_params[12][16][3]={
{
{-0.004921, -0.401732, -1.0},
{-0.056081, -0.858201, -1.0},
{-0.025654, 0.484709, -1.0},
{0.011628, 0.350071, -1.0},
{0.272988, -1.240794, -1.0},
{0.337926, -1.606079, -1.0},
{-0.177881, 1.501004, -1.0},
{-0.152147, 1.109596, -1.0},
{0.372826, -1.482309, -1.0},
{0.5, -1.520448, -1.0},
{-0.313956, 1.863906, -1.0},
{-0.222068, 1.471548, -1.0},
{0.289207, -1.096345, -1.0},
{0.465006, -1.458004, -1.0},
{-0.101701, 1.253563, -1.0},
{0.150533, 0.599672, -1.0},
},
{
{-0.020121, -0.323824, -1.0},
{0.071318, -1.066257, -1.0},
{-0.026655, 0.476101, -1.0},
{0.03393, 0.226471, -1.0},
{0.242702, -1.24198, -1.0},
{0.217093, -1.374719, -1.0},
{-0.206476, 1.549646, -1.0},
{-0.289544, 1.257647, -1.0},
{0.398272, -1.593794, -1.0},
{0.499996, -1.615652, -1.0},
{-0.270708, 1.819482, -1.0},
{-0.284096, 1.423998, -1.0},
{0.156795, -0.870569, -1.0},
{0.49007, -1.472816, -1.0},
{0.012478, 0.973464, -1.0},
{-0.02701, 0.924094, -1.0},
},
{
{-0.105538, -0.129568, -1

In [18]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dphi1 = array( 'd' )
dphi_err1 = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
dynamic_dphi1 = []
dynamic_dphi_err1 = []
for iphi in range(N_phi):
    if not is_sigmalised and not new: dphi.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
    if new and not is_sigmalised: dphi.append( round(0.002*phi_bins[2*iphi]-0.001-0.1,5))
    if is_sigmalised: 
        dphi.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
        dphi1.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
    dphi_err.append(0.001/4)
    dphi_err1.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    dynamic_dphi_ifile1 = []
    dynamic_dphi_err_ifile1 = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        dynamic_dphi_ilayer1 = array( 'd' )
        dynamic_dphi_err_ilayer1 = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(max(0.05,fit_mean_functions[ifile][ilayer][iphi].GetParError(0)))
            dynamic_dphi_ilayer1.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(1))
            dynamic_dphi_err_ilayer1.append(max(0.05,fit_mean_functions[ifile][ilayer][iphi].GetParError(1)))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
        dynamic_dphi_ifile1.append(dynamic_dphi_ilayer1)
        dynamic_dphi_err_ifile1.append(dynamic_dphi_err_ilayer1)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)
    dynamic_dphi1.append(dynamic_dphi_ifile1)
    dynamic_dphi_err1.append(dynamic_dphi_err_ifile1)

print("dphi = ", dphi)
print(dynamic_dphi)
print(dynamic_dphi1)

dphi =  array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1])
[[array('d', [0.02538978676383261, -0.09323082150253915, 0.022398076002585678, -0.004921114458869007, -0.02808985281621876, -0.04274610913201701]), array('d', [0.23129480539007768, 0.02995505051052782, -0.05608142131520275, 0.12933289732937742, -0.10400376500627008, -0.03143745052728959]), array('d', [-0.253623972758003, -0.10326811667564915, 0.013868210906974912, -0.02565362253186612, -0.05535917233224882, 0.1708144214331984]), array('d', [-0.0072129397513486715, 0.016918475743688162, 0.011627621868295553, -0.055392314105462866, 0.035738886077113424, -0.2773570508637252]), array('d', [0.2489868815090269, 0.12884095847021088, 0.14377155800213592, 0.2729878626216153, 0.2909129077763976, 0.4714279555230068]), array('d', [0.30021853358623657, 0.275407429963817, 0.33792580062194066, 0.2967464551612571, 0.4999817176017257, 0.49905857280789423]), array('d', [-0.16513728849106887, -0.20964630987432112, -0.11334958695606473, -0.177881299022

In [19]:
g_mean_phi = []
for ifile in range(N_files):
    c4 = root.TCanvas(f"c4{ifile}",f"c4{ifile}",600*2,450)
    c4.Divide(2,1)
    g_mean_phi_type=[]
    h3=[]
    for itype in range(2):
        g_mean_phi_layer=[]
        c4.cd(itype+1)
        Min, Max, x0, x1 = -0.05, 0.05, -0.02, 0.05
        if is_sigmalised==1: Max, Min, x0, x1 = 1+itype*2, -1-itype*2, -2, 5
        names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
        legends =[]
        for iphi in range(1):
            h3.append(Format_Pad_old( x0, x1, Min, Max, "phi_prev [rad]", "mean [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
            h3[-1].Draw()
            legends.append(root.TLegend(0.55,0.4,0.85,0.9))
            legends[-1].SetFillColorAlpha(0,0)
            legends[-1].SetLineWidth(0)
            for ilayer in range(0,N_layers):

                if itype==0:                
                    g_mean_phi_layer.append(root.TGraphErrors(N_phi, dphi, dynamic_dphi[ifile][ilayer],dphi_err,dynamic_dphi_err[ifile][ilayer]))
                    g_mean_phi_layer[-1].SetName(f"test_graph_{itype}_{ifile}_{ilayer}_{iphi}_{iOption0}")
                else:
                    g_mean_phi_layer.append(root.TGraphErrors(N_phi, dphi1, dynamic_dphi1[ifile][ilayer],dphi_err1,dynamic_dphi_err1[ifile][ilayer]))

                Format_Graph(g_mean_phi_layer[-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
                g_mean_phi_layer[-1].Fit("pol1","Q","")
                g_mean_phi_layer[-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
                g_mean_phi_layer[-1].Draw('sameP')
                legends[-1].AddEntry(g_mean_phi_layer[-1],layers[ilayer],"p")

            legends[-1].Draw()
        g_mean_phi_type.append(g_mean_phi_layer)
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_phi_{ifile}{iOption0}{is_sigmalised}.png")
    g_mean_phi.append(g_mean_phi_type)
print(g_mean_phi[0][0][0].GetFunction("pol1").GetParameter(0),g_mean_phi[0][1][0].GetFunction("pol1").GetParameter(0))


-0.020693625455009433 -0.4353668538586232


In [20]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_functions[ifile][ilayer][iphi].Eval(pt_point)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
            dynamic_dphi_err_ilayer.append((fit_functions[ifile][ilayer][iphi].GetParError(0)*2)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [1.029863238539603, 1.1229866726647386, 1.012063751362015, 0.9879362486379852, 1.105423340727507, 1.0744434580566347]), array('d', [1.0241271352355843, 1.0787563333343573, 1.0296791133523469, 0.9703208866476528, 1.1264155107399414, 1.1484686593489626]), array('d', [1.0651902375546398, 1.0621790996544687, 0.997022857132034, 1.002977142867966, 0.9793245215558678, 0.9594417018165512]), array('d', [1.0870833033587768, 1.063970283267309, 1.048279614411481, 0.9517203855885188, 1.0485963859038636, 0.9628890534300383]), array('d', [0.7861220598407254, 0.005061564623333539, 1.0076255103827247, 0.9923744896172753, 0.745467283159057, 0.9813742438112248]), array('d', [1.0714175696862571, 1.0543035743685718, 0.9970506153114417, 1.0029493846885582, 1.3378593750250503, 1.083891360711302]), array('d', [0.7163905901953681, 1.2026043949343608, 0.8357907647728517, 1.1642092352271483, 0.968649623835617, 1.121905565411264]), array('d', [0.7347682373601195, 1.0681133853071345, 1.004725841125208

In [21]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_phi]*N_layers]*N_files
    Min, Max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: Max, Min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, Min, Max, "phi_prev [rad]", "sigma [rad]", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, sigma_dynamic_dphi[ifile][ilayer],dphi_err,sigma_dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_sigma_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma_phi[ifile][ilayer][iphi][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][ilayer][iphi][-1].GetFunction("pol2").SetLineColor(colors[ilayer%4])
            g_sigma_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/sigma_phi_{ifile}{iOption0}{is_sigmalised}.png")

In [22]:
if False:
    print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_layers}][3]="+"\n{")
    for fit_functions_ifile in fit_functions:
        print("    {")
        for fits in fit_functions_ifile:
            print("        {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
        print("    },")      
    print("};")
    print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][3]="+"\n{")
    for ifile in range(N_files):
        print("    {")
        for ilayer in range(N_layers):
            print("        {"+f'{round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(0),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(1),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(2),6)}'+"},")
        print("    },")
    print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params0[{N_layers}][{N_files}][2]="+"\n{")
for ilayer in range(N_layers):
    print("    {")
    for ifile in range(N_files):
        print("        {"+f'{round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")

print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params1[{N_layers}][{N_files}][2]="+"\n{")
for ilayer in range(N_layers):
    print("    {")
    for ifile in range(N_files):
        print("        {"+f'{round(g_mean_phi[ifile][1][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][1][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")

const float phi_mean_phi_params0[16][12][2]=
{
    {
        {-0.020694, -0.007868},
        {-0.059815, 0.01114},
        {-0.021526, -0.032052},
        {-0.060633, -0.028005},
        {-0.082318, -0.006431},
        {0.076919, -0.021275},
        {0.052248, -0.072775},
        {0.138593, -0.177466},
        {-0.00899, -0.107325},
        {0.107515, -0.109048},
        {0.094423, -0.120194},
        {0.000801, 0.009179},
    },
    {
        {0.028059, -0.082069},
        {0.000202, -0.106303},
        {-0.031299, -0.10528},
        {0.015838, -0.152934},
        {0.038113, -0.15925},
        {-0.039018, -0.111687},
        {0.004066, -0.159749},
        {0.00828, -0.131527},
        {-0.009615, -0.086312},
        {0.023979, -0.159734},
        {-0.130913, -0.032227},
        {-0.206559, -0.009527},
    },
    {
        {-0.034286, 0.113438},
        {-0.068425, -0.017555},
        {-0.053642, -0.085729},
        {-0.054742, -0.037586},
        {-0.037201, -0.022107},
        {0.015

In [23]:
outrootfile = root.TFile(f"output/dynamic_delta_fits/dynamicdeltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_layer in g_sigma_f:
        for g_sigma_phi in g_sigma_layer:
            g_sigma_phi.Write()